DATA PREP 
1. Imports

In [13]:

import os
import sys
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.append(project_root)

import cv2, os, numpy as np, time, joblib
from tqdm.notebook import tqdm_notebook
from skimage.feature import local_binary_pattern, hog
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold 
# i asked ai about this and it suggested me this type of kfold. 
# i will provide some info abt it in the latter markdown
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from utils.prep_general import (
    load_and_preprocess_data, extract_features, split_data, scale_features, apply_pca, get_config, get_feature_params
)
from utils.prep_svm import (
    train_svm, tune_svm_hyperparameters, evaluate_model
)
print("test")

test


-> https://aistudio.google.com/prompts/new_chat 
    Why StratifiedKFold instead of standard KFold?

        StratifiedKFold: This is similar to KFold, but it makes the splits by preserving the percentage of samples for each class. If your overall dataset has 75% class A and 25% class B, StratifiedKFold will ensure that each of the k folds also has approximately 75% class A and 25% class B samples. Each fold is roughly the same size and has a representative class distribution.

        Why StratifiedKFold for classification (especially with imbalance)?

            Reliable Evaluation: In classification problems, especially with imbalanced datasets (like yours, where Fire=1 is the minority class at ~24% in the Kaggle data), a standard KFold might accidentally create one or more folds that have very few or even zero samples from the minority class. If a fold used for validation has no fire images, the model's performance (like Recall or F1-score) on that fold will be undefined or misleadingly high/low.
            Robust Training: Similarly, if a fold used for training has very few fire images, the model might not learn to identify the minority class effectively from that fold.
            StratifiedKFold avoids this by ensuring each fold is a miniature version of the overall dataset's class distribution. This means the performance metric calculated on each fold is a more reliable estimate, and the average metric across all folds (which GridSearchCV uses) is a better overall performance estimate for your hyperparameter tuning.

    In your project's context: Your Kaggle dataset has class imbalance (755 fire, 244 non-fire). Using StratifiedKFold for the cross-validation within GridSearchCV ensures that when GridSearchCV tests different hyperparameter combinations, it gets a stable and realistic F1-score (or whichever scoring metric you choose) for each combination by evaluating on folds that accurately represent the imbalance you expect the model to face. If you used standard KFold, the F1-scores from different folds could vary wildly just due to random chance in how the minority class samples were distributed across folds.



-> https://www.datacamp.com/tutorial/k-fold-cross-validation
Stratified Group K-Fold Cross-Validation 	Primarily classification with grouped data 	Combines stratification and group integrity, ensuring that groups are not split across folds. 	Great for grouped and imbalanced datasets to maintain both class and group integrity.
just before conclusion there is a table for kfold types, i took the info from there

2. Loading & Extracting Features
TODO: pca yi unuttum

In [ ]:
dataset_choice = 'kaggle' 
try:
    config = get_config(dataset_choice)
except ValueError as e:
    print(e)
    config = None
feature_params = get_feature_params()

if config:
    if dataset_choice == 'kaggle':
        config['data_root'] = os.path.join('..', 'fire_dataset') # todo change : os.path.join('..', 'data_subsets', 'fire_dataset')
    elif dataset_choice == 'dfire' and config['split_name'] == 'train':
        config['data_root'] = os.path.join('..', 'D-Fire', 'train') # todo also change (add data_subsets)

processed_data_dict = None
if config:
    processed_data_dict = load_and_preprocess_data(config)
    if processed_data_dict is None or processed_data_dict.get('labels', []).shape[0] == 0:
        processed_data_dict = None

features_array = np.array([])
labels_array = np.array([])
if processed_data_dict:
    features_array, labels_array = extract_features(processed_data_dict, feature_params)
    if features_array.shape[0] == 0:
        features_array = np.array([])

Using dataset: kaggle
Data root: fire_dataset
Target image size: (128, 128)
Color spaces loaded: ['bgr', 'hsv', 'ycbcr']
Normalize pixels: True

Feature extraction parameters: {'hist_bins': 100, 'lbp_radius': 3, 'lbp_n_points': None, 'lbp_method': 'uniform', 'hog_orientations': 9, 'hog_pixels_per_cell': (8, 8), 'hog_cells_per_block': (2, 2), 'hog_block_norm': 'L2-Hys'}

Kaggle data_root güncellendi: ..\fire_dataset


Processing fire_images: 100%|██████████| 755/755 [00:07<00:00, 102.47it/s]



toplam islenen: 755 atlanan: 0


Processing non_fire_images: 100%|██████████| 244/244 [00:04<00:00, 60.87it/s]



toplam islenen: 998 atlanan: 1

Successfully loaded and preprocessed 998 images.


Extracting Features:   0%|          | 0/998 [00:00<?, ?it/s]C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skimage\feature\texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
Extracting Features: 100%|██████████| 998/998 [00:16<00:00, 59.98it/s]


total featurei cikarilabilmis resim s.: 998
feature arr shape: (998, 8626)
label arr shape: (998,)


3. Data Split & Scale

In [15]:
X_train_orig, X_test_orig, y_train, y_test = None, None, None, None
if features_array.shape[0] > 0:
    X_train_orig, X_test_orig, y_train, y_test = split_data(features_array, labels_array)
    
X_train_pca, X_test_pca, pca_model = None, None, None
if X_train_orig is not None:
    n_pca_components = 0.95
    #X_train_pca, X_test_pca, pca_model = apply_pca(X_train_orig, X_test_orig, n_components=n_pca_components)

X_train_for_scaling = X_train_pca if X_train_pca is not None else X_train_orig
X_test_for_scaling = X_test_pca if X_test_pca is not None else X_test_orig

X_train_scaled, X_test_scaled, scaler = None, None, None
if X_train_for_scaling is not None:
    X_train_scaled, X_test_scaled, scaler = scale_features(X_train_for_scaling, X_test_for_scaling)


training split: (80%) testing split: (20%)
training features shape: (798, 8626)
testing features shape: (200, 8626)
training labels shape: (798,)
testing labels shape: (200,)


4. Model Training & Eval

In [16]:
initial_svm_model = None
if X_train_scaled is not None:
    initial_svm_model = train_svm(X_train_scaled, y_train, C=1.0, gamma='scale', kernel='rbf')
    if initial_svm_model:
        evaluate_model(initial_svm_model, X_test_scaled, y_test, model_name="SVM")

svm model training suresi: 3.00 saniye
Accuracy: 0.8900
Precision: 0.9006
Recall (Sensitivity): 0.9603
F1 Score: 0.9295

Confusion Matrix (SVM):
[[ 33  16]
 [  6 145]]

class report (SVM):
              precision    recall  f1-score   support

           0       0.85      0.67      0.75        49
           1       0.90      0.96      0.93       151

    accuracy                           0.89       200
   macro avg       0.87      0.82      0.84       200
weighted avg       0.89      0.89      0.89       200



5. HP Tuning

In [17]:
tuned_grid_search = None
if X_train_scaled is not None:
    param_grid_svm = {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.01, 0.1, 1],
        'kernel': ['rbf', 'linear']
    }
    tuned_grid_search = tune_svm_hyperparameters(X_train_scaled, y_train, param_grid_svm)
    if tuned_grid_search:
        best_svm_model_tuned = tuned_grid_search.best_estimator_
        evaluate_model(best_svm_model_tuned, X_test_scaled, y_test, model_name="Tuned SVM")

Fitting 5 folds for each of 40 candidates, totalling 200 fits

GridSearchCV 194.47 saniye surdu

en iyi parametreler:
{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}

en iyi cv f1 skoru:
0.9448913558489259
Accuracy: 0.9100
Precision: 0.9346
Recall (Sensitivity): 0.9470
F1 Score: 0.9408

Confusion Matrix (Tuned SVM):
[[ 39  10]
 [  8 143]]

class report (Tuned SVM):
              precision    recall  f1-score   support

           0       0.83      0.80      0.81        49
           1       0.93      0.95      0.94       151

    accuracy                           0.91       200
   macro avg       0.88      0.87      0.88       200
weighted avg       0.91      0.91      0.91       200



6. Save Models

In [18]:
MODEL_SAVE_DIR = os.path.join('..', 'models')
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
model_filename = 'best_svm_model_pca.pkl' if pca_model is not None else 'best_svm_model.pkl'
scaler_filename = 'scaler_pca.pkl' if pca_model is not None else 'scaler.pkl'
pca_filename = 'pca_model.pkl' if pca_model is not None else None 

MODEL_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, model_filename)
SCALER_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, scaler_filename)
PCA_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, pca_filename) if pca_filename else None

if tuned_grid_search is not None and scaler is not None: 
   best_svm_model_to_save = tuned_grid_search.best_estimator_
   try: joblib.dump(best_svm_model_to_save, MODEL_SAVE_PATH)
   except Exception as e: pass

   try: joblib.dump(scaler, SCALER_SAVE_PATH)
   except Exception as e: pass

   if pca_model is not None and PCA_SAVE_PATH is not None:
        try: joblib.dump(pca_model, PCA_SAVE_PATH)
        except Exception as e: pass